In [6]:
import * as React from "react";
import { useState, useEffect, useCallback } from "react";
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  shorthands,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";

const useStyles = makeStyles({
  container: {
    display: "flex",
    flexDirection: "column",
    width: "100%",
    height: "100vh",
    backgroundColor: tokens.colorNeutralBackground3,
    ...shorthands.padding("24px"),
    boxSizing: "border-box",
    overflow: "hidden",
  },
  actionBar: {
    display: "flex",
    flexDirection: "column",
    width: "100%",
    backgroundColor: tokens.colorNeutralBackground2,
    borderRadius: tokens.borderRadiusMedium,
    transition: "all 0.2s ease",
    boxShadow: tokens.shadow4,
    cursor: "pointer",
    ":hover": {
      backgroundColor: tokens.colorNeutralBackground2Hover,
      transform: "translateY(-2px)",
      boxShadow: tokens.shadow8,
    },
  },
  actionContent: {
    display: "flex",
    flexDirection: "column",
    gap: "8px",
    padding: "16px",
  },
  actionTitle: {
    fontSize: tokens.fontSizeBase500,
    fontWeight: tokens.fontWeightSemibold,
    color: tokens.colorNeutralForeground1,
  },
  actionDescription: {
    fontSize: tokens.fontSizeBase300,
    color: tokens.colorNeutralForeground3,
  },
  iconWrapper: {
    display: "flex",
    alignItems: "center",
    justifyContent: "center",
    width: "32px",
    height: "32px",
    borderRadius: "50%",
    backgroundColor: "#106ebe",
    color: tokens.colorNeutralBackground1,
    marginRight: "12px",
  },
  chatPage: {
    position: "absolute",
    top: 0,
    left: 0,
    right: 0,
    bottom: 0,
    backgroundColor: tokens.colorNeutralBackground1,
    display: "flex",
    flexDirection: "column",
    padding: "24px",
  },
  chatMessages: {
    flexGrow: 1,
    overflowY: "auto",
    display: "flex",
    flexDirection: "column",
    gap: "16px",
    marginBottom: "16px",
  },
  chatMessage: {
    padding: "12px",
    borderRadius: tokens.borderRadiusMedium,
    maxWidth: "70%",
  },
  userMessage: {
    alignSelf: "flex-end",
    backgroundColor: tokens.colorBrandBackground,
    color: tokens.colorNeutralForegroundInverted,
  },
  assistantMessage: {
    alignSelf: "flex-start",
    backgroundColor: tokens.colorNeutralBackground2,
    color: tokens.colorNeutralForeground1,
  },
  chatInput: {
    display: "flex",
    gap: "8px",
  },
  chatInputField: {
    flex: 1,
    padding: "8px",
    borderRadius: tokens.borderRadiusMedium,
    border: `1px solid ${tokens.colorNeutralStroke1}`,
  },
  statusContainer: {
    display: "flex",
    justifyContent: "center",
    alignItems: "center",
    marginTop: "24px",
    minHeight: "60px",
  },
  statusMessage: {
    padding: "12px 24px",
    borderRadius: tokens.borderRadiusMedium,
    backgroundColor: tokens.colorNeutralBackground1,
    color: tokens.colorNeutralForeground1,
    boxShadow: tokens.shadow4,
    textAlign: "center",
    fontSize: tokens.fontSizeBase300,
    fontWeight: tokens.fontWeightSemibold,
    display: "flex",
    flexDirection: "column",
    alignItems: "center",
    justifyContent: "center",
    maxWidth: "80%",
  },
  successMessage: {
    backgroundColor: tokens.colorSuccessBackground,
    color: tokens.colorSuccessForeground1,
  },
  errorMessage: {
    backgroundColor: tokens.colorErrorBackground,
    color: tokens.colorErrorForeground1,
  },
});

const EmailGenerator = () => {
  const styles = useStyles();
  const [isLoading, setIsLoading] = useState(true);
  const [isProcessing, setIsProcessing] = useState(false);
  const [userConfig, setUserConfig] = useState(null);
  const [statusMessage, setStatusMessage] = useState(null);
  const [error, setError] = useState(null);
  const [chatMessages, setChatMessages] = useState([]);
  const [chatInput, setChatInput] = useState("");
  const [isChatActive, setIsChatActive] = useState(false);

  const fetchUserId = useCallback(async () => {
    try {
      const response = await fetch("http://localhost:8001/getUserId");
      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const data = await response.json();
      return data.userId;
    } catch (e) {
      console.error("Error fetching user ID:", e);
      setError("Failed to load user ID. Please try again.");
      return null;
    }
  }, []);

  const fetchUserConfig = useCallback(async (userId) => {
    setIsLoading(true);
    setError(null);
    try {
      console.log(`Fetching user configuration for user: ${userId}`);
      const response = await fetch(`http://localhost:8001/getUserConfig/${userId}`);
      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const data = await response.json();
      console.log("Received user configuration:", data);
      setUserConfig(data);
    } catch (e) {
      console.error("Error fetching user configuration:", e);
      setError(`Failed to load user configuration: ${e.message}`);
    } finally {
      setIsLoading(false);
    }
  }, []);

  useEffect(() => {
    const initializeComponent = async () => {
      const userId = await fetchUserId();
      if (userId) {
        await fetchUserConfig(userId);
      }
    };
    initializeComponent();
  }, [fetchUserId, fetchUserConfig]);

  const getEmailContent = async () => {
    return new Promise((resolve, reject) => {
      Office.context.mailbox.item.body.getAsync(Office.CoercionType.Text, (result) => {
        if (result.status === Office.AsyncResultStatus.Succeeded) {
          resolve(result.value);
        } else {
          reject(new Error(result.error.message));
        }
      });
    });
  };

  const wrapInHtml = (content) => {
    if (content == null) {
      console.error("Received null or undefined content in wrapInHtml");
      return "<html><body><p>Error: No content available.</p></body></html>";
    }

    const processMarkdown = (text) => {
      if (typeof text !== 'string') {
        console.error("Non-string content passed to processMarkdown:", text);
        return "";
      }
      
      // Process block elements
      text = text
        // Convert headers
        .replace(/^(#{1,6})\s+(.*?)$/gm, (match, hashes, content) => {
          const level = hashes.length;
          return `<h${level}>${content}</h${level}>`;
        })
        // Convert lists
        .replace(/(\n|^)(\s*)(\d+\.|-)\s+(.+)/g, (match, lineStart, indent, bullet, item) => {
          const listType = bullet === '-' ? 'ul' : 'ol';
          return `${lineStart}<${listType}><li>${item}</li></${listType}>`;
        })
        // Handle nested lists
        .replace(/<\/(ul|ol)>\n(<\1>)/g, '')
        .replace(/<\/li>\n(<li>)/g, '</li>$1')
        // Convert paragraphs
        .replace(/(\n|^)(?!<\/?(ul|ol|li|h[1-6]|p|a|code|pre|strong|em|hr))(.+)/g, '$1<p>$3</p>');

      // Process inline elements
      text = text
        // Convert links
        .replace(/\[([^\]]+)\]\(([^)]+)\)/g, '<a href="$2">$1</a>')
        // Convert bold text
        .replace(/\*\*(.*?)\*\*/g, '<strong>$1</strong>')
        // Convert italic text
        .replace(/\*(.*?)\*/g, '<em>$1</em>')
        // Convert code blocks
        .replace(/```(\w+)?\n([\s\S]*?)```/g, '<pre><code>$2</code></pre>')
        // Convert inline code
        .replace(/`([^`]+)`/g, '<code>$1</code>')
        // Convert horizontal rules
        .replace(/^---$/gm, '<hr>');

      return text;
    };

    try {
      const processedContent = processMarkdown(content);
      
      return `
        <!DOCTYPE html>
        <html lang="en">
          <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <style>
              body {
                font-family: Arial, sans-serif;
                font-size: 11pt;
                color: #000;
                max-width: 800px;
                margin: 0 auto;
                padding: 20px;
              }
              h1, h2, h3, h4, h5, h6 { margin-top: 1em; margin-bottom: 0.5em; }
              p { margin-top: 0; margin-bottom: 1em; }
              pre { background-color: #f4f4f4; padding: 10px; border-radius: 4px; overflow-x: auto; }
              code { background-color: #f4f4f4; padding: 2px 4px; border-radius: 2px; font-family: monospace; }
              ul, ol { padding-left: 30px; margin-bottom: 1em; }
              li { margin-bottom: 0.5em; }
              hr { border: none; border-top: 1px solid #ccc; margin: 20px 0; }
              a { color: #0066cc; text-decoration: none; }
              a:hover { text-decoration: underline; }
            </style>
          </head>
          <body>
            <div class="email-content">${processedContent}</div>
          </body>
        </html>
      `;
    } catch (error) {
      console.error("Error in wrapInHtml:", error);
      return "<html><body><p>Error processing content.</p></body></html>";
    }
  };

  const handleActionBarClick = () => {
    setIsChatActive(true);
  };

  const handleAction = async (userInput) => {
    setIsProcessing(true);
    setError(null);
    setStatusMessage(null);
    try {
      const action = userConfig.buttons[0];
      console.log(`Handling action: ${action.label}, User input: ${userInput}`);
      const content = await getEmailContent();
      const payload = {
        userId: userConfig.userId,
        emailContent: content,
        prompt: userInput,
      };

      console.log(`Sending request to: ${action.apiEndpoint}`);
      const response = await fetch(action.apiEndpoint, {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify(payload),
      });

      if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
      const responseData = await response.json();

      console.log("Received response:", responseData);
      const wrappedContent = wrapInHtml(responseData.body || responseData.generatedContent);
      setChatMessages(prevMessages => [
        ...prevMessages,
        { role: "assistant", content: wrappedContent }
      ]);
    } catch (e) {
      console.error(`Error in handleAction: ${e.message}`);
      setError(`Failed to process your request. Please try again.`);
    } finally {
      setIsProcessing(false);
    }
  };

  const handleChatSubmit = (e) => {
    e.preventDefault();
    if (chatInput.trim()) {
      setChatMessages(prevMessages => [...prevMessages, { role: "user", content: chatInput }]);
      handleAction(chatInput);
      setChatInput("");
    }
  };

  if (isLoading) {
    return (
      <div className={styles.container}>
        <div className={styles.statusContainer}>
          <Spinner size="large" label="Loading user data..." />
        </div>
      </div>
    );
  }

  if (error) {
    return (
      <div className={styles.container}>
        <div className={styles.statusContainer}>
          <div className={`${styles.statusMessage} ${styles.errorMessage}`}>
            {error}
          </div>
        </div>
      </div>
    );
  }

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <div
              key={index}
              className={`${styles.chatMessage} ${
                message.role === "user" ? styles.userMessage : styles.assistantMessage
              }`}
            >
              {message.role === "user" ? (
                message.content
              ) : (
                <div dangerouslySetInnerHTML={{ __html: message.content }} />
              )}
            </div>
          ))}
        </div>
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  return (
    <div className={styles.container}>
      <div className={styles.actionBar} onClick={handleActionBarClick}>
        <div className={styles.actionContent}>
          <div style={{ display: "flex", alignItems: "center" }}>
            <div className={styles.iconWrapper}>
              <FluentIcons.MailTemplate24Regular />
            </div>
            <Text className={styles.actionTitle}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.actionDescription}>{userConfig.buttons[0].description}</Text>
        </div>
      </div>
    </div>
  );
};

export default EmailGenerator;

ModuleNotFoundError: No module named 'streamlit'